<a href="https://colab.research.google.com/github/chrispi21/python-dataeng/blob/main/06_programowanie_obiektowe_i_obsluga_wyjatkow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programowanie obiektowe

Docs:
1. https://realpython.com/python3-object-oriented-programming/
2. https://realpython.com/python-classes/

Dla chętnych:
1. [YT: Protocol Or ABC In Python - When to Use Which One?
](https://www.youtube.com/watch?v=xvb5hGLoK0A)
2. [YT: Protocols vs ABCs in Python - When to Use Which One?](https://www.youtube.com/watch?v=dryNwWvSd4M)
3. [YT: Dependency Inversion: Write BETTER PYTHON CODE Part 2](https://www.youtube.com/watch?v=Kv5jhbSkqLE)
4. https://realpython.com/python-getter-setter/

## Co to jest klasa (`class`)? Co to jest obiekt (`object`)?

Klasa jest szablonem, który opisuje jak tworzyć obiekty (nazywane również instancjami klasy). Nazwa klasa określa byt, który ma reprezentować (rzeczownik). Klasa może zawierać informacje o cechach nazywane atrybutami. Może także zawierać metody, czyli funkcje odpowiedzialne za zachowania klasy.




In [ ]:
# Przykład najprostszej klasy
class NajprostszaKlasa:
  pass

In [ ]:
# instancja najprostszej klasy
NajprostszaKlasa()

W przypadku tworzenia bardziej złożonych klas wykorzystuje się specjalną metodę `__init__` nazywaną konstruktorem. Dzięki niej możemy tworzyć instancję na podstawie przekazanych argumentów. Zmienna `self` jest instancją tworzonego obiektu - dzięki temu możemy przypisać przekazane argumenty:

In [ ]:
class NajprostszaKlasaV2:
  def __init__(self, parametr1, parametr2):
    self.attr1 = parametr1
    self.attr2 = parametr2

In [ ]:
instancja_v2 = NajprostszaKlasaV2("arg1", "arg2")

In [ ]:
instancja_v2

In [ ]:
instancja_v2.attr1

In [ ]:
class NajprostszaKlasaV3:
  def __init__(self, parametr1, parametr2):
    self.attr1 = parametr1
    self.attr2 = parametr2

  def metoda(self, parametr_metody):
    # gdy odwołujemy się do atrybytutów, korzystamu z self
    print("attr1", self.attr1)
    # nie robimy tego w przypadku parametrów metod
    print("parametr_metody", parametr_metody)

In [ ]:
instancja = NajprostszaKlasaV3("arg1", "arg2")
instancja.metoda("Prosta metoda")

Domyślnie modyfikacje są dozwolone:

In [ ]:

instancja.attr1 = "zmieniony arg1"

In [ ]:
instancja.attr1

## Podstawy dziedziczenia

Dziedziczenie polega na przekazywaniu cech klas bazowych (klasa-rodzic; `base class`, `parent class`, `superclass`) klasom pochodnym (klasa-dziecko, `derived class`, `child class`, `subclass`).

Do przykładu będzie potrzebny nam plik jak na ostatnich zajęciach:

In [ ]:
!wget -O pracownicy.csv https://raw.githubusercontent.com/chrispi21/python-dataeng/refs/heads/main/pracownicy.csv

Utworzymy klasę-rodzica, która służy do odczytania pliku. Nasza klasa jeszcze nie będzie potrafiła odczytywać danych.

In [ ]:
from os.path import exists

class FileReader:
  def __init__(self, path):
    self.path = path

  def validate_path(self):
    return exists(self.path)

  def read(self):
    print("Nie umiem czytać :( ")

Zobaczmy jak to działa:

In [ ]:
file_reader = FileReader("/content/pracownicy.csv")

In [ ]:
file_reader.validate_path()

In [ ]:
file_reader.read()

Utworzymy teraz klasę dziecko. Klasa ta będzie odpowiedzialna za odczyta danych w formacie CSV:

In [ ]:
import pandas as pd

class CsvReader(FileReader):
  def read(self):
    return pd.read_csv(self.path)

Sprawdźmy jej działanie:

In [ ]:
csv_reader = CsvReader("/content/pracownicy.csv")

Dziecko potrafi to samo co rodzic:

In [ ]:
csv_reader.validate_path()

Potrafi też czytać dane w formacie CSV:

In [ ]:
csv_reader.read()

Do następnego przykładu będą nam potrzebne dane z duplikatami. Wygenerujmy plik zawierający duplikaty:

In [ ]:
!cat /content/pracownicy.csv >> /content/duplikaty_pracownicy.csv && tail -10 /content/pracownicy.csv >> /content/duplikaty_pracownicy.csv

Dodamy kolejną klasę-dziecko, która będzie odpowiedzialna za odczyt CSV i usuwanie duplikatów:

In [ ]:
class DistinctCsvReader(CsvReader):
  def read(self):
    # możemy odwołać się do klasy-rodzica za pomocą super()
    return super().read().drop_duplicates()

In [ ]:
file_reader = DistinctCsvReader("/content/duplikaty_pracownicy.csv")

Metoda zaimplementowana w `FileReader`:

In [ ]:
file_reader.validate_path()

Nowa implementacja metody `read`:

In [ ]:
file_reader.read()

Ćwiczenie

Korzystając z klasy rodzica `FileReader` utwórz klasę potomną, która służy do odczytu danych w formacie JSON (analogicznie jak `CsvReader` z powyższego przykładu).

Wersja dla chętnych:

* gdzie rzucać wyjątek `NotImplementedError` w klasie `FileReader`
* jak skorzystałbyś z klasy abstrakcyjnej do implementacji `FileReader`?

In [ ]:
# @title Rozwiązanie

In [ ]:
# @title Podpowiedź

import pandas as pd

class JsonReader(FileReader):
  def read(self):
    return pd.read_json(self.path, orient="records", lines=True)

# Przykładowe dane:
# dane_csv = pd.read_csv("/content/pracownicy.csv")
# dane_csv.to_json("/content/pracownicy.json", orient="records", lines=True)

JsonReader("/content/pracownicy.json").read()

# Obsługa wyjątków

Mechanizm obsługi wyjątków:
* chroni przed awariami programów
* pozwala na debugowanie i logowanie błędów
* umożliwia kontrolę nad wykonywamiem aplikacji w przypadku błędów

Docs:
1. https://docs.python.org/3.13/tutorial/errors.html

## Podstawowa składnia

Przeanalizujmy poniższe przykłady:

In [ ]:
try: # Kod, który chcemy wykonać w bezpieczny sposób
    dzielnik = int(input("Podaj dzielnik: "))
    wynik = 10 / dzielnik  # Jeśli dzielnik == 0, to ZeroDivisionError
except ValueError: # Kod wykonywany, gdy nastąpi ValueError
    print("Podaj liczbę!")
except ZeroDivisionError: # Kod wykonywany, gdy nastąpi ZeroDivisionError
    print("Nie można dzielić przez 0")
else:
    print(f"Wynik: {wynik}") # Tylko gdy nie ma błędu
finally:
    print("Koniec.")  # Zawsze


Obsługa wielu wyjątków jednocześnie:

In [ ]:
try:
    dzielnik = int(input("Podaj dzielnik: "))
    wynik = 10 / dzielnik
except (ValueError, ZeroDivisionError) as e: # Możemy obsłużyć wiele wyjątków jednocześnie
    print(f"Wyjątek: {e}. Podaj liczbę różną od 0!")
else:
    print(f"Wynik: {wynik}")
finally:
    print("Koniec.")

Rzucanie wyjątków:

In [ ]:
wiek = int(input("Tylko dla pełnoletnich! Ile masz lat? "))
if wiek < 18:
  raise ValueError("Brak dostępu!") #
print("Zapraszamy!")

Własne klasy wyjątków:

In [ ]:
class AgeRestrictedContentException(Exception):
  pass

wiek = int(input("Tylko dla pełnoletnich! Ile masz lat? "))
if wiek < 18:
  raise AgeRestrictedContentException("Brak dostępu!")
print("Zapraszamy!")

Dla chętnych - bardziej zwięzły sposób:

In [ ]:
from contextlib import suppress

with suppress(ZeroDivisionError):
  print("Dzielę przez 0!")
  1/0
  print("Wiem, że to się nie uda :(")

print("Koniec!")

## Ćwiczenie

### Etap I - refaktoring

*zadanie będzie traktowane tak samo jak aktywność*

Zrób refactoring poniższego kodu:

```python
import pandas as pd

def extract(path):
  return pd.read_csv(path)

def transform(df):
  return df[["Imię"]].drop_duplicates()

def load(df, path):
  return df.to_json(path, orient="records", index=False, lines=True)


def job(input_path, output_path):
  source_data = extract(input_path)
  transformed_data = transform(source_data)
  load(transformed_data, output_path)
```

1. Utwórz klasę `CsvExtractor` z metodą `extract`. Parametry metody `extract`: `path`.
2. Utwórz klasę `Deduplicator` z metodą `transform`. Konstruktor klasy przyjmuje listę pól na bazie, których odbywa się deduplikacja. `transform` nie posiada parametrów.
3. Utwórz klasę `JsonLoader` z metodą `load`. Konstruktor przyjmuje parametry: `orient`, `index`, `lines`. Metoda `load` przyjmuje `path`.
4. Utwórz klasę `Job`. Parametry konstruktora: `input_path`, `output_path` oraz obiekty typu: `CsvExtractor`, `Deduplicator`, `JsonLoader`. Metoda `run` nie posiada parametrów, ale odpowiada za uruchomienie job'a.
5. Utwórz plik `main.py`, który tworzy instancję `Job` i uruchamia metodę `run`.

Dobrze korzystać z modułów i pakietów.

Rozwiązanie najlepiej umieścić na Github albo w ramach usługi uczelnianej.

Linki do rozwiązań proszę wysłać mi na priv na MS Teams. Najlepiej jak będą to Pull Request'y z feature branch'a do branch'a main/master - ułatwi to ew. komentowanie.

*pozostała część dla chętnych*

### Etap II - zmiana wymagań 1
1. Dodaj klasę `ParquetLoader`, która zapisuje dane w formacie parquet.
2. Utwórz plik `main_parquet.py`, który tworzy instancję `Job` i uruchamia metodę `run`.

Jak pracuje się Tobie z modularnym kodem?

### Etap III - zmiana wymagań 2
Tym razem potrzebny nam jest nowe przetwarzanie, które zlicza pracowników w poszczególnych departamentach. Postaram się wykorzystać klasy `CsvExtractor`, `ParquetLoader`, `Job`. Utwórz plik  `main_departments.py`.

Jak pracuje się Tobie z modularnym kodem?

### Etap IV

Spróbuj skorzystać z `typing`: https://realpython.com/python-type-checking/

### Etap V
W zasobach dot. tych zajęć znajdują się filmy dot. klas abstrakcyjnych, protokołów i `dependency inversion`. Zaproponuj hierarchę klas / protokoły (w zależności co wybierzesz), które najlepiej pasowałyby do naszego przypadku.

### Etap VI
Za pomocą jednego z frameworków do testów:
1. https://docs.pytest.org/en/stable/
2. https://docs.python.org/3/library/unittest.html

Stwórz klasę testową, która zawsze zwraca te wiersze nie odczytując ich z pliku. Wiersze do zwrócenia są argumentami konstruktora. Parametr `path` metody `extract` nie ma wpływu na zwrócony wynik. Utwórz przypadek testowy, który obejmuje:
1. Utworzenie instancji nowej klasy
2. Utworzenie instancji klasy `Job` bazując na nowej klasie
3. Sprawdzeniu, czy dla danych testowych zwracane są dane zgodnie z oczekiwanymi

Zakaz korzystania z mock'ów.
